## Tutorial: Getting started with torch-neuron (resnet-50 tutorial - compile steps in code)
**NOTE:** This notebook content represents the compilation parts of the [getting started tutorial](./getting_started.md) - it is not intended to used without reference to the tutorial.  This is why we start at step 3 below :).

Steps Overview:

We'll cover the following topics as we progress:

1. How do I analyze a model for use with AWS Neuron?
2. How can I compile for different batch sizes to maximise the throughput of my model on neuron hardware?



### Step 3: Compile on compilation instance

The following cell runs a compilation with a batch size of 1 on the torchvision resnet-50 model (pretrained)



In [1]:
!pip install torchvision==0.8.2

In [2]:
import torch
import numpy as np
import os
import torch_neuron
from torchvision import models
import logging

## Enable logging so we can see any important warnings
logger = logging.getLogger('Neuron')
logger.setLevel(logging.INFO)

image = torch.zeros([1, 3, 224, 224], dtype=torch.float32)

## Load a pretrained ResNet50 model
model = models.resnet50(pretrained=True)

## Tell the model we are using it for evaluation (not training)
model.eval()

## Analyze the model - this will show operator support and operator count
torch.neuron.analyze_model( model, example_inputs=[image] )

## Now compile the model - with logging set to "info" we will see
## what compiles for Neuron, and if there are any fallbacks
model_neuron = torch.neuron.trace(model, example_inputs=[image])

## Export to saved model
model_neuron.save("resnet50_neuron.pt")

INFO:Neuron:The following operations are currently supported in torch-neuron for this model:
INFO:Neuron:aten::batch_norm
INFO:Neuron:aten::relu
INFO:Neuron:prim::Constant
INFO:Neuron:aten::_convolution
INFO:Neuron:aten::addmm
INFO:Neuron:aten::max_pool2d
INFO:Neuron:aten::add
INFO:Neuron:aten::flatten
INFO:Neuron:aten::adaptive_avg_pool2d
INFO:Neuron:aten::t
INFO:Neuron:prim::ListConstruct
INFO:Neuron:100.00% of all operations (including primitives) (1698 of 1698) are supported
INFO:Neuron:100.00% of arithmetic operations (176 of 176) are supported
INFO:Neuron:All operators are compiled by neuron-cc (this does not guarantee that neuron-cc will successfully compile)
INFO:Neuron:Number of arithmetic operators (pre-compilation) before = 176, fused = 176, percent fused = 100.0%
INFO:Neuron:compiling function _NeuronGraph$1110 with neuron-cc
INFO:Neuron:Compiling with command line: '/home/ubuntu/anaconda3/envs/aws_neuron_pytorch_p36/bin/neuron-cc compile /tmp/tmpqyc7i_t_/graph_def.pb --fra

You should see something like this, if your instance is correctly configured (your output may vary in some details):

```
INFO:Neuron:The following operations are currently supported in torch-neuron for this model:
INFO:Neuron:aten::relu
INFO:Neuron:aten::flatten
INFO:Neuron:aten::t
INFO:Neuron:aten::max_pool2d
INFO:Neuron:aten::add
INFO:Neuron:aten::addmm
INFO:Neuron:aten::_convolution
INFO:Neuron:aten::batch_norm
INFO:Neuron:aten::adaptive_avg_pool2d
INFO:Neuron:prim::ListConstruct
INFO:Neuron:prim::Constant
INFO:Neuron:100.00% of all operations (including primitives) (1645 of 1645) are supported
INFO:Neuron:100.00% of arithmetic operations (176 of 176) are supported
OrderedDict([('percent_supported', 100.0), ('percent_supported_arithmetic', 100.0), ('supported_count', 1645), ('total_count', 1645), ('supported_count_arithmetic', 176), ('total_count_arithmetic', 176), ('supported_operators', {'aten::relu', 'aten::flatten', 'aten::t', 'aten::max_pool2d', 'aten::add', 'aten::addmm', 'aten::_convolution', 'aten::batch_norm', 'aten::adaptive_avg_pool2d', 'prim::ListConstruct', 'prim::Constant'}), ('unsupported_operators', []), ('operators', ['aten::_convolution', 'aten::adaptive_avg_pool2d', 'aten::add', 'aten::addmm', 'aten::batch_norm', 'aten::flatten', 'aten::max_pool2d', 'aten::relu', 'aten::t', 'prim::Constant', 'prim::ListConstruct']), ('operator_count', OrderedDict([('aten::_convolution', 53), ('aten::adaptive_avg_pool2d', 1), ('aten::add', 16), ('aten::addmm', 1), ('aten::batch_norm', 53), ('aten::flatten', 1), ('aten::max_pool2d', 1), ('aten::relu', 49), ('aten::t', 1), ('prim::Constant', 1252), ('prim::ListConstruct', 217)]))])
INFO:Neuron:Number of arithmetic operators (pre-compilation) before = 176, fused = 176, percent fused = 100.0%
INFO:Neuron:compiling function _NeuronGraph$1108 with neuron-cc
INFO:Neuron:Compiling with command line: '/home/ubuntu/test_beta_env/bin/neuron-cc compile /tmp/tmp2fisdcmu/graph_def.pb --framework TENSORFLOW --pipeline compile SaveTemps --output /tmp/tmp2fisdcmu/graph_def.neff --io-config {"inputs": {"0:0": [[1, 3, 224, 224], "float32"]}, "outputs": ["Add_69:0"]}''
INFO:Neuron:Number of arithmetic operators (post-compilation) before = 176, compiled = 176, percent compiled = 100.0%
```



You can find more on torch.neuron.analyze_model and torch.neuron.trace using the following commands

In [3]:
import torch
import torch.neuron

help(torch.neuron.analyze_model)
help(torch.neuron.trace)

Help on function analyze_model in module torch_neuron.model_support:

analyze_model(model, example_inputs)
    Analyze a PyTorch model for use with AWS Neuron
    
    Parameters
    ----------
    model : callable python function or a class derived from torch.nn.Module
        The input PyTorch model to analyze
    example_inputs : torch.Tensor or list of torch.Tensor or tuple of torch.Tensor
        A list or tuple of input tensors used to trace the model and create a model graph for analysis
    
    Returns
    -------
    dict
        Dictionary containing names and counts of present, neuron supported and non-neuron-supported 
        operations
    
    Notes
    -----
    The purpose of this function is to provide information on the expected coverage of a model using 
    the AWS neuron ahead of time compiler.  This is intended to give users an idea of how well
    a model will translate, and whether key operations are not supported

Help on function trace in module torch_neuron

3.3 WARNING: If you run the inference script (in section 4 below) on your CPU instance you will get output, but see the following warning.

```
[E neuron_op_impl.cpp:53] Warning: Tensor output are *** NOT CALCULATED *** during CPU
execution and only indicate tensor shape
```

The warning is also displayed during trace (where it is expected).  It is not a concern that you see this during compilation

This is an artifact of the way we trace a model on your compile instance. 

NB: Do not perform inference with a neuron traced model on a non neuron supported instance, results will not be calculated, and this warning will be shown


3.4. If not compiling and inferring on the same instance, copy the compiled artifacts to the inference server:

```
scp -i <PEM key file>  ./resnet50_neuron.pt ubuntu@<instance DNS>:~/ # if Ubuntu-based AMI
scp -i <PEM key file>  ./resnet50_neuron.pt ec2-user@<instance DNS>:~/  # if using AML2-based AMI
```



## Step 7: Experiment with different batch sizes:

Now that we are using all four cores we can experiment with compiling and running large batch sizes on each of our four cores.  Here will will provide the compilation code for batch 5


In [4]:
import torch
import numpy as np
import os
import torch_neuron 
from torchvision import models

## Enable logging so we can see any important warnings
logger = logging.getLogger('Neuron')
logger.setLevel(logging.INFO)

batch_size = 5

image = torch.zeros([batch_size, 3, 224, 224], dtype=torch.float32)

## Load a pretrained ResNet50 model
model = models.resnet50(pretrained=True)

## Tell the model we are using it for evaluation (not training)
model.eval()

## Analyze the model - this will show operator support and operator count
torch.neuron.analyze_model( model, example_inputs=[image] )

## Now compile the model
model_neuron = torch.neuron.trace(model, example_inputs=[image])

## Export to saved model
model_neuron.save("resnet50_neuron_b{}.pt".format(batch_size))

INFO:Neuron:The following operations are currently supported in torch-neuron for this model:
INFO:Neuron:aten::batch_norm
INFO:Neuron:aten::relu
INFO:Neuron:prim::Constant
INFO:Neuron:aten::_convolution
INFO:Neuron:aten::addmm
INFO:Neuron:aten::max_pool2d
INFO:Neuron:aten::add
INFO:Neuron:aten::flatten
INFO:Neuron:aten::adaptive_avg_pool2d
INFO:Neuron:aten::t
INFO:Neuron:prim::ListConstruct
INFO:Neuron:100.00% of all operations (including primitives) (1698 of 1698) are supported
INFO:Neuron:100.00% of arithmetic operations (176 of 176) are supported
INFO:Neuron:All operators are compiled by neuron-cc (this does not guarantee that neuron-cc will successfully compile)
INFO:Neuron:Number of arithmetic operators (pre-compilation) before = 176, fused = 176, percent fused = 100.0%
INFO:Neuron:compiling function _NeuronGraph$2222 with neuron-cc
INFO:Neuron:Compiling with command line: '/home/ubuntu/anaconda3/envs/aws_neuron_pytorch_p36/bin/neuron-cc compile /tmp/tmpf9l9qvj9/graph_def.pb --fra

7.2. If not compiling and inferring on the same instance, copy the compiled artifacts to the inference server:

```
scp -i <PEM key file>  ./resnet50_neuron_b5.pt ubuntu@<instance DNS>:~/ # if Ubuntu-based AMI
scp -i <PEM key file>  ./resnet50_neuron_b5.pt ec2-user@<instance DNS>:~/  # if using AML2-based AMI
```